In [1]:
from collections import Counter

# -----------------------------
# 3.1 MANUAL BPE DEMO
# -----------------------------

toy_corpus = [
    "low", "low", "low", "low", "low",
    "lowest", "lowest",
    "newer", "newer", "newer", "newer", "newer", "newer",
    "wider", "wider", "wider",
    "new", "new"
]

def tokenize_word(word):
    return list(word) + ["_"]

tokenized_toy = [tokenize_word(w) for w in toy_corpus]

def get_stats(corpus):
    pairs = Counter()
    for word in corpus:
        for i in range(len(word) - 1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_corpus(corpus, pair):
    new_corpus = []
    replacement = "".join(pair)
    for word in corpus:
        new_word, i = [], 0
        while i < len(word):
            if i < len(word)-1 and (word[i], word[i+1]) == pair:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_corpus.append(new_word)
    return new_corpus

print("\n=== 3.1 Manual BPE: First 3 merges ===")
manual_tokenized = tokenized_toy
vocab = set(ch for word in manual_tokenized for ch in word)

for step in range(3):
    stats = get_stats(manual_tokenized)
    best = max(stats, key=stats.get)
    manual_tokenized = merge_corpus(manual_tokenized, best)
    vocab.add("".join(best))
    print(f"Step {step+1}: Merge {best} -> {''.join(best)} (count={stats[best]})")
    print("Updated vocab:", vocab)
    print("Corpus snippet:", manual_tokenized[:3], "\n")


# -----------------------------
# 3.2 MINI BPE LEARNER
# -----------------------------

print("\n=== 3.2 Mini-BPE on Toy Corpus ===")
tokenized = [tokenize_word(w) for w in toy_corpus]
merges = []

for step in range(5):  # Learn 5 merges
    stats = get_stats(tokenized)
    best = max(stats, key=stats.get)
    merges.append(best)
    tokenized = merge_corpus(tokenized, best)
    print(f"Step {step+1}: Merge {best} -> {''.join(best)} (count={stats[best]})")

def segment(word, merges):
    word = list(word) + ["_"]
    for a, b in merges:
        i = 0
        while i < len(word)-1:
            if word[i] == a and word[i+1] == b:
                word[i:i+2] = ["".join([a, b])]
            else:
                i += 1
    return word

words = ["new", "newer", "lowest", "widest", "newestest"]
print("\nSegmentation results:")
for w in words:
    print(w, "->", segment(w, merges))


# -----------------------------
# 3.3 BPE ON PARAGRAPH
# -----------------------------

print("\n=== 3.3 BPE on Paragraph ===")
paragraph = """
The students were learning natural language processing.
They practiced tokenization and subword segmentation.
Newer methods improved accuracy in wider contexts.
However, the smallest datasets posed the lowest challenges.
Machine learning models adapted quickly.
Deep learning created powerful new systems.
"""

para_corpus = [tokenize_word(w.lower()) for w in paragraph.split()]

# Train 30 merges
merges_para = []
tokenized_para = para_corpus
for step in range(30):
    stats = get_stats(tokenized_para)
    if not stats: break
    best = max(stats, key=stats.get)
    merges_para.append(best)
    tokenized_para = merge_corpus(tokenized_para, best)

# Show top 5 merges
print("Top 5 merges:", merges_para[:5])

# Longest 5 subwords
subwords = set(ch for word in tokenized_para for ch in word)
longest = sorted(subwords, key=len, reverse=True)[:5]
print("Longest 5 subwords:", longest)

# Segment some words
words_para = ["new", "newer", "lowest", "segmentation", "deepest"]
print("\nSegmentation on paragraph words:")
for w in words_para:
    print(w, "->", segment(w, merges_para))

# Reflection
print("\nReflection:")
print("BPE learned subwords like suffixes (-ing, -est), stems (learn, process), and whole words (new).")
print("It solves the OOV problem: unseen words like 'deepest' are split into known parts (deep + est).")
print("Sometimes splits match morphemes (er_, est_), sometimes they are arbitrary (segmen+ta+tion).")
print("Pros: handles rare words, reduces vocabulary size. Cons: loses readability, can break morphemes unnaturally.")
print("Overall, subword tokenization balances word-level and character-level models efficiently.")



=== 3.1 Manual BPE: First 3 merges ===
Step 1: Merge ('e', 'r') -> er (count=9)
Updated vocab: {'i', 'n', 'l', 'w', 't', '_', 'er', 's', 'd', 'o', 'e', 'r'}
Corpus snippet: [['l', 'o', 'w', '_'], ['l', 'o', 'w', '_'], ['l', 'o', 'w', '_']] 

Step 2: Merge ('er', '_') -> er_ (count=9)
Updated vocab: {'i', 'n', 'l', 'w', 't', '_', 'er_', 'er', 's', 'd', 'o', 'e', 'r'}
Corpus snippet: [['l', 'o', 'w', '_'], ['l', 'o', 'w', '_'], ['l', 'o', 'w', '_']] 

Step 3: Merge ('n', 'e') -> ne (count=8)
Updated vocab: {'i', 'n', 'l', 'ne', 'w', 't', '_', 'er_', 'er', 's', 'd', 'o', 'e', 'r'}
Corpus snippet: [['l', 'o', 'w', '_'], ['l', 'o', 'w', '_'], ['l', 'o', 'w', '_']] 


=== 3.2 Mini-BPE on Toy Corpus ===
Step 1: Merge ('e', 'r') -> er (count=9)
Step 2: Merge ('er', '_') -> er_ (count=9)
Step 3: Merge ('n', 'e') -> ne (count=8)
Step 4: Merge ('ne', 'w') -> new (count=8)
Step 5: Merge ('l', 'o') -> lo (count=7)

Segmentation results:
new -> ['new', '_']
newer -> ['new', 'er_']
lowest -> ['lo', 